In [1]:
import os
import numpy as np
import pywt
import random
import seaborn as sns
import tensorflow as tf
from keras.layers import Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (
  Input, Conv1D, BatchNormalization, Activation, GlobalAveragePooling1D, 
  Dense, Dropout, GRU, Concatenate, LayerNormalization, MultiHeadAttention, 
  Reshape, Multiply, Softmax
)
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from utils import (
  encode_labels, check_gpu_availability, plot_loss_accuracytupian, 
  evaluate_model, plot_confusion_matrixtupian, plot_tsne, 
  plot_precision_recall_curve_multiclasstupian, plot_roc_curve_multiclasstupian, 
  AdjustLearningRateCallback, denoise2,count_labels,denoise2_iterative2,AdjustLearningRateCallback
)
from utils import plot_precision_recall_curve_multiclass,plot_roc_curve_multiclass2,calculate_g_mean,plot_confusion_matrix,plot_confusion_matrix2,plot_loss_accuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1DTranspose


c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
check_gpu_availability()

GPU 可用


In [3]:
datafilename1 = "C:\\Users\\Administrator\\Desktop\\database\\cinc2017denoise.npz"
data1 = np.load(datafilename1, allow_pickle=True)
X_train, y_train, X_val, y_val, X_test, y_test = data1['ecgstrain'], data1['labelstrain'], data1['ecgsval'], data1['labelsval'], data1['ecgstest'], data1['labelstest']

In [4]:
y_train = encode_labels(y_train)
y_test = encode_labels(y_test)
y_val= encode_labels(y_val)
y_train = to_categorical(y_train, num_classes=4)
y_val=to_categorical(y_val, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [5]:
from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, Concatenate

def ince_block(inputs, filters, strides=1):
    x1 = Conv1D(filters, 3, strides=strides, padding='same')(inputs)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)

    x2 = Conv1D(filters, 5, strides=1, padding='same')(inputs)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)

    x3 = Conv1D(filters, 9, strides=1, padding='same')(inputs)
    x3 = BatchNormalization()(x3)
    x3 = Activation('relu')(x3)

    x4 = Conv1D(filters, 17, strides=1, padding='same')(inputs)
    x4 = BatchNormalization()(x4)
    x4 = Activation('relu')(x4)

    x = Concatenate()([x1, x2, x3, x4])
    x = Activation('relu')(x)
    return x


In [6]:
from tensorflow.keras.layers import GlobalAveragePooling1D, Reshape, Dense, Multiply
def se_block(input_tensor, reduction_ratio=16):
  channels = input_tensor.shape[-1]
  x = GlobalAveragePooling1D()(input_tensor)
  x = Reshape((1, channels))(x)
  x = Dense(channels // reduction_ratio, activation='relu', use_bias=False)(x)
  x = Dense(channels, activation='sigmoid', use_bias=False)(x)
  x = Multiply()([input_tensor, x])
  return x


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dropout, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size, strides):
    shortcut = x
    # 主卷积路径
    x = Conv1D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # 如果shortcut的形状与x不匹配，则调整shortcut
    if shortcut.shape[-1] != x.shape[-1] or shortcut.shape[-2] != x.shape[-2]:
        shortcut = Conv1D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # 添加跳过连接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 3, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = residual_block(x, 32, 3, 2)
    x = residual_block(x, 64, 3, 2)
    x = residual_block(x, 128,3, 2)
    x = residual_block(x, 256,3, 2)
    x=GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Example usage:
model =resnet((4500, 1), 4)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4500, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 2250, 32)     128         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 2250, 32)     128         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 2250, 32)     0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [8]:
callback = AdjustLearningRateCallback(factor=0.1, patience=2, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
history=model.fit(X_train, y_train, batch_size=256, epochs=80, validation_data=(X_val, y_val), callbacks=[callback,early_stopping])

Epoch 1/80
55/55 [==============================] - 15s 84ms/step - loss: 0.3874 - accuracy: 0.6147 - val_loss: 0.4201 - val_accuracy: 0.6082
Epoch 2/80
55/55 [==============================] - 4s 69ms/step - loss: 0.3323 - accuracy: 0.6851 - val_loss: 0.3736 - val_accuracy: 0.6185
Epoch 3/80
55/55 [==============================] - 4s 69ms/step - loss: 0.3126 - accuracy: 0.7097 - val_loss: 0.3484 - val_accuracy: 0.6371
Epoch 4/80
55/55 [==============================] - 4s 69ms/step - loss: 0.3004 - accuracy: 0.7292 - val_loss: 0.3308 - val_accuracy: 0.6768
Epoch 5/80
55/55 [==============================] - 4s 69ms/step - loss: 0.2872 - accuracy: 0.7413 - val_loss: 0.3126 - val_accuracy: 0.7052
Epoch 6/80
55/55 [==============================] - 4s 69ms/step - loss: 0.2803 - accuracy: 0.7508 - val_loss: 0.3231 - val_accuracy: 0.6872
Epoch 7/80
55/55 [==============================] - 4s 69ms/step - loss: 0.2732 - accuracy: 0.7599 - val_loss: 0.3040 - val_accuracy: 0.7286
Epoch 8/80
5

In [9]:
import numpy as np

def add_low_frequency_noise_multidim(data, snr, frequency_range=(0, 5), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [10]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,0)

In [11]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.6651386549303698
Recall: 0.39438269768747225
F1 Score: 0.25718014433827807
Accuracy: 0.6138374899436846
Class 1 - Precision: 1.0, Recall: 0.004405286343612335, F1 Score: 0.008771929824561405
Class 2 - Precision: 0.6304898648648649, Recall: 0.9946702198534311, F1 Score: 0.7717756526234169
Class 3 - Precision: 0.9130434782608695, Recall: 0.028455284552845527, F1 Score: 0.05519053876478318
Class 4 - Precision: 0.11702127659574468, Recall: 0.55, F1 Score: 0.1929824561403509
Class 1 Accuracy: 0.9090909090909091
Class 2 Accuracy: 0.6448109412711183
Class 3 Accuracy: 0.7107803700724055
Class 4 Accuracy: 0.9629927594529365


In [12]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,5)

In [13]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.6713140482435181
Recall: 0.3938880556235792
F1 Score: 0.38120711998375334
Accuracy: 0.6383748994368463
Class 1 - Precision: 0.6875, Recall: 0.09691629955947137, F1 Score: 0.1698841698841699
Class 2 - Precision: 0.6316455696202532, Recall: 0.9973351099267155, F1 Score: 0.7734435546370446
Class 3 - Precision: 0.8955223880597015, Recall: 0.08130081300813008, F1 Score: 0.14906832298136646
Class 4 - Precision: 0.47058823529411764, Recall: 0.4, F1 Score: 0.4324324324324324
Class 1 Accuracy: 0.913515687851971
Class 2 Accuracy: 0.6472244569589702
Class 3 Accuracy: 0.7244569589702333
Class 4 Accuracy: 0.9915526950925181


In [14]:
import numpy as np

def add_high_frequency_noise_multidim(data, snr, frequency_range=(0, 200), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [15]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,0)

In [16]:
evaluate_model(model,X_test_noisy,y_test)

Precision: 0.4331820556353663
Recall: 0.35526478282770835
F1 Score: 0.26157758166473544
Accuracy: 0.6122284794851166
Class 1 - Precision: 0.0, Recall: 0.0, F1 Score: 0.0
Class 2 - Precision: 0.618693134822167, Recall: 0.9966688874083944, F1 Score: 0.763460066343455
Class 3 - Precision: 0.9473684210526315, Recall: 0.024390243902439025, F1 Score: 0.04755614266842801
Class 4 - Precision: 0.16666666666666666, Recall: 0.4, F1 Score: 0.23529411764705882
Class 1 Accuracy: 0.9082864038616251
Class 2 Accuracy: 0.6271118262268705
Class 3 Accuracy: 0.7099758648431215
Class 4 Accuracy: 0.9790828640386162


In [17]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,5)

In [18]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.7334765676882268
Recall: 0.41867329852079477
F1 Score: 0.4263217528270107
Accuracy: 0.6468222043443282
Class 1 - Precision: 0.7631578947368421, Recall: 0.1277533039647577, F1 Score: 0.21886792452830187
Class 2 - Precision: 0.6363249680986814, Recall: 0.9966688874083944, F1 Score: 0.7767393561786087
Class 3 - Precision: 0.891566265060241, Recall: 0.1002710027100271, F1 Score: 0.1802679658952497
Class 4 - Precision: 0.6428571428571429, Recall: 0.45, F1 Score: 0.5294117647058824
Class 1 Accuracy: 0.916733708769107
Class 2 Accuracy: 0.6540627514078842
Class 3 Accuracy: 0.7292839903459373
Class 4 Accuracy: 0.9935639581657281
